# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [49]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import pickle

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import re

from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [31]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from matplotlib import pyplot as plt 

import re
import pickle
import time
import datetime

from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.multioutput import MultiOutputClassifier
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer

In [50]:
# load data from database
engine = create_engine('sqlite:///Massages.db')
df = pd.read_sql("clean_msg", engine)
X = df['message']
y = df.drop(['message', 'genre', 'id', 'original'], axis=1)

In [51]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26177 entries, 0 to 26176
Data columns (total 40 columns):
id                        26177 non-null int64
message                   26177 non-null object
original                  10153 non-null object
genre                     26177 non-null object
related                   26177 non-null int64
request                   26177 non-null int64
offer                     26177 non-null int64
aid_related               26177 non-null int64
medical_help              26177 non-null int64
medical_products          26177 non-null int64
search_and_rescue         26177 non-null int64
security                  26177 non-null int64
military                  26177 non-null int64
child_alone               26177 non-null int64
water                     26177 non-null int64
food                      26177 non-null int64
shelter                   26177 non-null int64
clothing                  26177 non-null int64
money                     26177 non-null i

In [54]:
# Check size of dataset
print("Complete feature dataset size = ", X.shape, "; complete target variables size = ", y.shape)

Complete feature dataset size =  (26177,) ; complete target variables size =  (26177, 36)


In [56]:
# Finally, save the names of the category labels
category_names = list(df.columns[4:])

In [57]:
# Check categories
category_names

['related',
 'request',
 'offer',
 'aid_related',
 'medical_help',
 'medical_products',
 'search_and_rescue',
 'security',
 'military',
 'child_alone',
 'water',
 'food',
 'shelter',
 'clothing',
 'money',
 'missing_people',
 'refugees',
 'death',
 'other_aid',
 'infrastructure_related',
 'transport',
 'buildings',
 'electricity',
 'tools',
 'hospitals',
 'shops',
 'aid_centers',
 'other_infrastructure',
 'weather_related',
 'floods',
 'storm',
 'fire',
 'earthquake',
 'cold',
 'other_weather',
 'direct_report']

### 2. Write a tokenization function to process your text data

In [38]:
stop_words = nltk.corpus.stopwords.words("english")
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()

def tokenize(text):
    """
    Description: 
        For a dataframe with massages, do following procedures.
        - Normalization to lowercase.
        - Remove punctuation characters.
        - Tokenization,lemmatization, and stop word removal.
        
    Arguments:
        text: text
    Returns:
        tokens: a list of tockenized text.
    """
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize andremove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return tokens

In [58]:
# Test function for the first 5 messages
for message in X[:5]:
    tokens = tokenize(message)
    print(message)
    print(tokens, '\n')

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'update', 'a', 'cold', 'front', 'from', 'cuba', 'that', 'could', 'pas', 'over', 'haiti'] 

Is the Hurricane over or is it not over
['is', 'the', 'hurricane', 'over', 'or', 'is', 'it', 'not', 'over'] 

Looking for someone but no name
['looking', 'for', 'someone', 'but', 'no', 'name'] 

UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['un', 'report', 'leogane', '80', '90', 'destroyed', 'only', 'hospital', 'st', 'croix', 'functioning', 'need', 'supply', 'desperately'] 

says: west side of Haiti, rest of the country today and tonight
['say', 'west', 'side', 'of', 'haiti', 'rest', 'of', 'the', 'country', 'today', 'and', 'tonight'] 



### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [59]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('forest', MultiOutputClassifier( RandomForestClassifier() ))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [60]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state =123, test_size =0.2)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [61]:

y_train.columns[0]

'related'

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [62]:
y_pred = pipeline.predict(X_test)

In [63]:
y_pred_t = np.transpose(y_pred)
y_pred_t.shape

(36, 5236)

In [64]:
i = 0
while i<len(y_pred_t):
    print(y_train.columns[i])
    print(classification_report(y_test[y_train.columns[i]], y_pred_t[i]))
    i = i+1

related
             precision    recall  f1-score   support

          0       0.63      0.39      0.48      1244
          1       0.83      0.93      0.88      3992

avg / total       0.78      0.80      0.78      5236

request
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      4334
          1       0.83      0.39      0.53       902

avg / total       0.88      0.88      0.86      5236

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5212
          1       0.00      0.00      0.00        24

avg / total       0.99      1.00      0.99      5236

aid_related
             precision    recall  f1-score   support

          0       0.74      0.87      0.80      3144
          1       0.73      0.55      0.63      2092

avg / total       0.74      0.74      0.73      5236

medical_help
             precision    recall  f1-score   support

          0       0.93      0.99      0

### 6. Improve your model
Use grid search to find better parameters. 

In [65]:
parameters = {
    'tfidf__norm': ['l1', 'l2'],
    'tfidf__sublinear_tf': [True, False] 
}

cv = GridSearchCV(pipeline, param_grid=parameters, cv=3, verbose=3, n_jobs=-1)

In [66]:
cv.fit(X_train, y_train)
cv.cv_results_

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] tfidf__norm=l1, tfidf__sublinear_tf=True ........................
[CV]  tfidf__norm=l1, tfidf__sublinear_tf=True, score=0.2257556224036671, total=  43.2s
[CV] tfidf__norm=l1, tfidf__sublinear_tf=True ........................


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   52.8s remaining:    0.0s


[CV]  tfidf__norm=l1, tfidf__sublinear_tf=True, score=0.22607449856733525, total=  43.8s
[CV] tfidf__norm=l1, tfidf__sublinear_tf=True ........................


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  1.8min remaining:    0.0s


[CV]  tfidf__norm=l1, tfidf__sublinear_tf=True, score=0.22664756446991405, total=  44.4s
[CV] tfidf__norm=l1, tfidf__sublinear_tf=False .......................
[CV]  tfidf__norm=l1, tfidf__sublinear_tf=False, score=0.2144391920928234, total=  42.6s
[CV] tfidf__norm=l1, tfidf__sublinear_tf=False .......................
[CV]  tfidf__norm=l1, tfidf__sublinear_tf=False, score=0.22464183381088826, total=  43.0s
[CV] tfidf__norm=l1, tfidf__sublinear_tf=False .......................
[CV]  tfidf__norm=l1, tfidf__sublinear_tf=False, score=0.22363896848137535, total=  43.0s
[CV] tfidf__norm=l2, tfidf__sublinear_tf=True ........................
[CV]  tfidf__norm=l2, tfidf__sublinear_tf=True, score=0.22002578427159433, total=  43.0s
[CV] tfidf__norm=l2, tfidf__sublinear_tf=True ........................
[CV]  tfidf__norm=l2, tfidf__sublinear_tf=True, score=0.21991404011461319, total=  43.0s
[CV] tfidf__norm=l2, tfidf__sublinear_tf=True ........................
[CV]  tfidf__norm=l2, tfidf__sublinear

[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed: 10.4min finished


{'mean_fit_time': array([ 39.13534371,  38.41550167,  38.39903402,  38.00806801]),
 'std_fit_time': array([ 0.6464684 ,  0.15655749,  0.14707592,  0.31305559]),
 'mean_score_time': array([ 4.65817078,  4.45167224,  4.48547673,  4.47570602]),
 'std_score_time': array([ 0.17900677,  0.0029327 ,  0.03499949,  0.06737481]),
 'param_tfidf__norm': masked_array(data = ['l1' 'l1' 'l2' 'l2'],
              mask = [False False False False],
        fill_value = ?),
 'param_tfidf__sublinear_tf': masked_array(data = [True False True False],
              mask = [False False False False],
        fill_value = ?),
 'params': [{'tfidf__norm': 'l1', 'tfidf__sublinear_tf': True},
  {'tfidf__norm': 'l1', 'tfidf__sublinear_tf': False},
  {'tfidf__norm': 'l2', 'tfidf__sublinear_tf': True},
  {'tfidf__norm': 'l2', 'tfidf__sublinear_tf': False}],
 'split0_test_score': array([ 0.22575562,  0.21443919,  0.22002578,  0.22518264]),
 'split1_test_score': array([ 0.2260745 ,  0.22464183,  0.21991404,  0.22277937]

In [68]:

y_pred2 = cv.predict(X_test)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [69]:
y_pred_t2 = np.transpose(y_pred2)
y_pred_t2.shape

(36, 5236)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [70]:
i = 0
while i<len(y_pred_t2):
    print(y_train.columns[i])
    print(classification_report(y_test[y_train.columns[i]], y_pred_t2[i]))
    i = i+1

related
             precision    recall  f1-score   support

          0       0.61      0.38      0.46      1244
          1       0.83      0.92      0.87      3992

avg / total       0.77      0.79      0.78      5236

request
             precision    recall  f1-score   support

          0       0.88      0.98      0.93      4334
          1       0.84      0.38      0.52       902

avg / total       0.88      0.88      0.86      5236

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5212
          1       0.00      0.00      0.00        24

avg / total       0.99      1.00      0.99      5236

aid_related
             precision    recall  f1-score   support

          0       0.75      0.88      0.81      3144
          1       0.75      0.55      0.63      2092

avg / total       0.75      0.75      0.74      5236

medical_help
             precision    recall  f1-score   support

          0       0.93      0.99      0

### 9. Export your model as a pickle file

In [71]:
with open("fitted_model.pickle", 'wb') as pickle_file:
    pickle.dump(cv, pickle_file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.